# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name: Emmanuel Macario
###### Student Number: 831659
###### Email: <macarioe@student.unimelb.edu.au>
###### Python version: 3.6.0

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

# Libraries Used

In [2]:
# Import useful libraries
from collections import defaultdict
import pandas as pd
import numpy as np

In [94]:
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'

# Preprocess Function

In [126]:

def preprocess(filename):
    """
    Opens a data file in csv, and transforms it into
    a usable format, in the form of a pandas DataFrame.
    """
    
    # Read csv data into a dataframe. Assign
    # an integer value to each attribute in
    # the data.
    df = pd.read_csv(filename, header=None)
        
    
    # Partition the data into a list of instances, and 
    # a list of class labels for those instances
    instance_list = df.iloc[:,:-1]
    class_list = df.iloc[:,-1]
    data_set = instance_list, class_list
    
    
    """
    # Give a description of the class distributions
    print("CLASS DISTRIBUTION")
    print(pd.Series(class_list).value_counts())
    
    print()
    print("ATTRIBUTE DISTRIBUTIONS")
    print(instance_list.describe())
    """
    
    
    return data_set


# Test preprocess function
instance_list, class_list = preprocess(CSV1)

print(pd.Series(class_list).value_counts())


def proportion_missing_values(instance_list):
    """
    Calculates and returns the proportion of 
    instances that contain at least one missing value
    in the entire dataset. Used for personal analysis 
    of the data.
    """
    
    return len(instance_list[instance_list.values == '?'])/len(instance_list)
    


no-recurrence-events    201
recurrence-events        85
Name: 9, dtype: int64


# Train Supervised Function

In [122]:

def train_supervised(instance_list, class_list):
    """
    Builds a supervised Naive Bays model,
    given a preprocessed set of training
    data, by calculating counts from the
    training data.
    
    Inputs: a training set of data, consisting
    of a list of instances, and a list of class 
    labels for those instances.
    
    Outputs: 2-tuple containing a class frequency 
    dictionary and also the supervised model.
    """
    
    # The supervised Naive Bays model
    supervised_model = {}
    
    # For every instance in the training
    # data, update the model
    for i in range(len(instance_list)):
        
        # Get the instance
        instance = instance_list.iloc[i,:]
        
        # Get the associated class label
        class_label = class_list[i]
        
        
        # If the class label is not already in
        # the model, create a nested set of
        # dictionaries for the class.
        if class_label not in supervised_model:
            
            # New dictionary for every new class
            supervised_model[class_label] = {}
            
            # For each attribute, initialise
            # a new default dictionary.
            for attr in instance_list.columns:
                supervised_model[class_label][attr] = defaultdict(int)
                
        
        # For every attribute in the instance,
        # get its corresponding value and update
        # the model.
        for attr in instance_list.columns:
            
            attr_value = instance[attr]
            
            # For missing values in a training instance, it is
            # possible to simply have them not contribute to the
            # attribute-value counts.
            if attr_value == '?':
                continue
                
            supervised_model[class_label][attr][attr_value] += 1
    
    
    return supervised_model



# Test function
supervised_model = train_supervised(instance_list, class_list)

# Predict Supervised Function

In [123]:

def predict_supervised(supervised_model, instance_list):
    """
    Predicts the class labels for a set of test
    instances, based on a supervised Naive Bayes
    model. Implements epsilon probabilistic smoothing.
    """
    
    # Epsilon value for epsilon-smoothing
    EPSILON = np.finfo(float).eps
    
    
    # The list of predictions for each instance
    prediction_list = []
    
    
    # Get the class frequencies to avoid redundant calculations
    class_freqs = get_class_freqs(supervised_model)
    
    
    # Calculate the total number of instances,
    total_instances = sum(class_freqs.values())
    
    
    # For each instance in the test set, predict 
    # its class based on the supervised model.
    for instance in instance_list.values:
        
        max_prob = 0.0
        predicted_class = ""
        
        # We obtain the probability of an instance
        # being a certain class, for each possible class.
        for class_label in class_freqs:
            
            # Firstly, get the prior probability of a class
            prob = class_freqs[class_label]/total_instances
            
            
            # Now, multiply the prior probability of the class
            # by the posterior probability of each of the instance's 
            # attribute values, given the class.
            for attr in supervised_model[class_label]:
                
                # Get the attribute value
                attr_value = instance[attr]
                
                # If an attribute value is missing for the training
                # instance, simply have it not count towards the probability
                # estimates
                if attr_value == '?':
                    continue
                
                # When accessing values, if value is 0, replace with epsilon
                post_attr = supervised_model[class_label][attr].get(attr_value, EPSILON)
                
                # Update the probability estimate
                prob *= (post_attr/class_freqs[class_label])
                
            
            # If the probability is the highest seen
            # thus far, set the predicted class to the
            # class label.
            if prob > max_prob:
                max_prob = prob
                predicted_class = class_label
            
        
        # Add the class label with the highest
        # corresponding probability to the list
        # of predictions.
        prediction_list.append(predicted_class)
        
        
    return prediction_list




def get_class_freqs(supervised_model):
    """
    Helper function that returns a dictionary containing 
    frequency counts of all the class labels in the training data.
    """
    
    class_freqs = defaultdict(int)
    
    for class_label in supervised_model:
        class_freqs[class_label] = sum(supervised_model[class_label][0].values())
        
    return class_freqs




# Test the function
prediction_list = predict_supervised(supervised_model, instance_list)

print(pd.Series(prediction_list).value_counts())

no-recurrence-events    209
recurrence-events        77
dtype: int64


# Evaluate Supervised Function

In [124]:

def evaluate_supervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in a supervised
    context. Uses accuracy as the primary method of
    evaluation.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    # Calculate and return the accuracy of the model
    return (prediction_list == class_list).value_counts().loc[True] / len(prediction_list)


# Test the function
evaluate_supervised(prediction_list, class_list)


0.7552447552447552

# Train Unsupervised Function

In [82]:

def train_unsupervised(instance_list, class_labels, class_distributions=None):
    """
    Builds a weak unsupervised Naive Bayes model,
    from a given set of unlabelled training data,
    and a unique set of possible class labels. This
    function is called by predict_unsupervised() to
    perform a single iteration to update the random 
    class distributions.
    """
    
    # If the random class distributions have not been
    # created, then for each training instance, create
    # a non-uniform (random) class distribution for 
    # that instance.
    if class_distributions is None:
        
        random_class_distributions = []

        for instance in instance_list.values:

            random_distribution = np.abs(np.random.randn(1, len(class_labels))[0])
            norm_random_distribution = random_distribution / sum(random_distribution)
            random_class_distributions.append(norm_random_distribution)

        # Store all random class distributions in a dataframe
        class_distributions = pd.DataFrame(random_class_distributions, 
                                           columns=class_labels)
        
        
        
    # Initialise the unsupervised Naive Bays model, 
    # which is just a set of nested dictionaries
    unsupervised_model = {c : {a : defaultdict(float) for a in instance_list.columns} for c in class_labels}
    
    
    # For every instance in the training set, update
    # fractional counts in the unsupervised model
    for i in range(len(instance_list)):
        
        for class_label in class_labels:
            
            for attr in instance_list.columns:
            
                attr_value = instance_list[attr][i]
                
                # If an attribute value is missing,
                # simply have it not contribute to counts
                if attr_value == '?':
                    continue
                
                unsupervised_model[class_label][attr][attr_value] += class_distributions[class_label][i]
                
    
    return class_distributions, unsupervised_model



# Test the function
class_labels = class_list.unique()
cd, um = train_unsupervised(instance_list, class_labels)
print([cd[class_label].sum() for class_label in cd.columns])

[443.85470744229787, 429.7178964935455, 428.46440011018655, 425.962995953971]


# Predict Unsupervised Function

In [40]:

def predict_unsupervised(class_distributions, unsupervised_model, instance_list, iterations=5):
    """
    Predicts the class distribution for a set of
    instances, based on a trained model. Delegates
    the task of updating class distributions in a
    single iteration to the train_unsupervised() function.
    Returns the final class distributions for instances,
    once all iterations have been completed.
    """
        
    # For each iteration, update the class distribution for each
    # instance, in an attempt to make them more reliable than
    # the initial random distribution.
    for iteration in range(iterations):
        
        # Calculate the frequency of the classes 
        # in the random class distribution
        class_freqs = get_class_freqs2(class_distributions)
         
        
        # Calculate the total number of instances
        total_instances = int(sum(class_freqs.values()) + 0.5)
    
        
        for i in range(len(instance_list)):

            instance = instance_list.iloc[i,:]

            for class_label in class_freqs:

                # Firstly, get the prior probability of a class
                prob = class_freqs[class_label] / total_instances

                # Now, multiply the prior probability of the class
                # by the posterior probability of each of the instance's 
                # attribute values, given the class.
                for attr in unsupervised_model[class_label]:
                    
                    attr_value = instance[attr]
                    
                    # Simply not have missing values count
                    # towards probability estimates.
                    if attr_value == '?':
                        continue
                    
                    prob *= (unsupervised_model[class_label][attr][attr_value] / class_freqs[class_label])


                # Update the class distribution for the instance
                class_distributions[class_label][i] = prob


        # Now, normalise the class distributions so that the 
        # probabilities add to 1 for every single instance.
        class_distributions = class_distributions.div(class_distributions.sum(axis=1), axis=0)
        

        
        # Update the unsupervised model based on the updated class distributions
        class_distributions, unsupervised_model = train_unsupervised(instance_list, 
                                                                     class_distributions.columns, 
                                                                     class_distributions)
        
        
    return class_distributions




def get_class_freqs2(class_distributions):
    """
    Helper function that returns a dictionary
    containing classes as keys, and the total
    sum of the fractional values for the class
    as values.
    """
    
    class_freqs = defaultdict(float)
    
    for class_label in class_distributions.columns:
        class_freqs[class_label] = class_distributions[class_label].sum()
        
    return class_freqs



# Test the function
class_distributions = predict_unsupervised(cd, um, instance_list)
class_distributions

,unacc,acc,vgood,good
0,0.258248,0.260228,0.222243,0.259281
1,0.246134,0.261741,0.223838,0.268288
2,0.251462,0.266696,0.223327,0.258515
3,0.259809,0.259314,0.223804,0.257073
4,0.247658,0.260859,0.225442,0.266041
5,0.252995,0.265772,0.224906,0.256327
6,0.258733,0.252528,0.220664,0.268075
7,0.246540,0.253938,0.222197,0.277325
8,0.251994,0.258866,0.221792,0.267347
9,0.268838,0.257342,0.225160,0.248659


# Evaluate Unsupervised Function

In [92]:

def evaluate_unsupervised(class_distributions, class_list):
    """
    Evaluates a set of predictions, in an
    unsupervised manner. Takes as input a set of 
    final class distributions for each instance, and 
    the actual class for each respective instance.
    """
    
    # Validation checking
    assert(len(class_distributions) == len(class_list))
    
    
    # The list of class predictions for the instances
    prediction_list = []
    
    
    # Assign the class with the highest probability in
    # the final distribution as the predicted class, and
    # add it to the list of predictions.
    for distribution in class_distributions.values:
        
        predicted_class = class_distributions.columns[np.argmax(distribution)]
        
        prediction_list.append(predicted_class)
            
    
    # Initialise and fill in the confusion matrix
    confusion_matrix = {predicted_class : defaultdict(int) for predicted_class in set(prediction_list)}
    
    for i in range(len(prediction_list)):
        predicted_class = prediction_list[i]
        actual_class = class_list[i]
        confusion_matrix[predicted_class][actual_class] += 1
    
    
    # Print the confusion matrix
    print("Confusion Matrix\n================") #Predicted |   Actual\n" + 100*"-")
    for predicted_class in confusion_matrix:
        print("{:20s}:".format(predicted_class) + "   {0}".format(confusion_matrix[predicted_class]))
    
    
    # Calculate the total number of 'correct' predictions. We
    # take the max value from each column in the confusion matrix,
    # and add it to the total number of correct predictions.
    total_correct = 0
    for predicted_class in confusion_matrix:
        total_correct += max(confusion_matrix[predicted_class].values())
    
    
    # Finally, calculate accuracy of the classifier
    accuracy = total_correct / len(prediction_list)
    
    
    return accuracy


# Test the function
evaluate_unsupervised(class_distributions, class_list)

Confusion Matrix
unacc               :   defaultdict(<class 'int'>, {'unacc': 331, 'acc': 79, 'good': 7, 'vgood': 14})
acc                 :   defaultdict(<class 'int'>, {'unacc': 401, 'acc': 118, 'vgood': 20, 'good': 24})
vgood               :   defaultdict(<class 'int'>, {'unacc': 236, 'acc': 128, 'vgood': 24, 'good': 28})
good                :   defaultdict(<class 'int'>, {'unacc': 242, 'acc': 59, 'vgood': 7, 'good': 10})


0.7002314814814815

# Driver Functions
These functions are drivers for both supervised and unsupervised Naive Bayes classifiers

In [116]:
# Driver for supervised Naive Bayes classification

def supervised_naive_bayes(filename):
    """
    Creates a supervised Naive Bayes model
    given a set of input data, classifies
    the instances based on the probabilistic 
    model, then prints the accuracy of the 
    classifier.
    """
    
    instance_list, class_list = preprocess(filename)
    supervised_model = train_supervised(instance_list, class_list)
    prediction_list = predict_supervised(supervised_model, instance_list)
    accuracy = evaluate_supervised(prediction_list, class_list)
    print(accuracy)
    
    
# Driver for unsupervised Naive Bayes classification
    
def unsupervised_naive_bayes(filename):
    """
    Creates a weak unsupervised Naive Bayes model
    given a set of training instances, classifies
    the instances by iteratively updating an initial
    set of random class distributions, then
    prints the accuracy of the classifier.
    """
    
    instance_list, class_list = preprocess(filename)
    class_labels = class_list.unique()
    class_distributions, unsupervised_model = train_unsupervised(instance_list, class_labels)
    final_class_distributions = predict_unsupervised(class_distributions, unsupervised_model, instance_list)
    accuracy = evaluate_unsupervised(final_class_distributions, class_list)
    print(accuracy)
    
    
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'


# Testing driver functions
filename = CSV3

unsupervised_naive_bayes(filename)


Confusion Matrix
hypothyroid         :   defaultdict(<class 'int'>, {'hypothyroid': 4, 'negative': 453})
negative            :   defaultdict(<class 'int'>, {'hypothyroid': 147, 'negative': 2559})
0.9522605121719886


Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.

# Discussion


### Question 1
Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.


Answer: _The unsupervised classifier works well on datasets that have both a small number of distinct classes, and a small number of distict values for their attributes. Furthermore, the classifier achieves lower variance in accuracy for datasets containing instances that are partitioned non-uniformly among classes._

_For example, the classifier shows high variance in accuracy for the mushroom data with a mean of 0.813, a low of 0.579 and a high of 0.895. This variance could be the result of many attributes having a high number of unique values, with a max of 12 unique values for attribute 8, and a mean of 5.32 unique values per attribute. Compounded with the fact that the training data is partitioned pretty evenly among two classes (4208 instances of 'e', and 3916 of 'p'), this could have made it difficult for the classifier to converge to a particular set of predictions, hence the randomness of results._

_Juxtaposed, consistent results are shown for the hypothyroid and breast cancer datasets, with mean accuracies of 0.952 and 0.703 respectively. 17 of 18 attributes in the hypothyroid data have only 2 possible unique values. A possible further reason for low variance could be that the distribution of instances among classes is highly non-uniform for both datasets. This could have a propensity to inflate accuracy, since accuracy is derived by taking the maximum value from each column in the confusion matrix._


**Note to assessor: corresponding accuracies for supervised NB are detailed in question two.**

### Question 2
When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.


Answer: _The accuracies of the supervised classifier across the datasets can be seen below:_

| Dataset        | Accuracy      |
| -------------  |:-------------:|
| _breast-cancer_| 0.755         | 
| _car_          | 0.874         |
| _hypothyroid_  | 0.952         |
| _mushroom_     | 0.997         |


_Since accuracy measures the ratio of correctly classified instances across all classes, this means that if one class occurs more than others, then the resulting accuracy is clearly dominated by the accuracy of the dominating class. This is the case for the hypothyroid dataset, since the classifier predicts 'negative' for all instances, when the training set is dominated by 'negative' instances._

_A surprising result was the accuracy of the classifier for the mushroom data. Such a high accuracy was not expected since 30.5% of instances contained at least one missing value. The reason a high accuracy was received is due to there being a large number of training instances containing a large distribution of possible attribute value-class pairs, and because Naive Bayes gracefully degrades when one or more predictor variables are missing or not observed. The latter is due to the implementation of epsilon smoothing and simply not having missing values contribute to frequency counts for their respective features._

_Compared to the other results, the accuracy of 0.755 for the breast cancer data was relatively low. Unlike the other datasets, the training set was relatively small, perhaps leading to a probabilistic model that did not cover a wide spectrum of attribute value-class pairs, possibly ultimately leading to a moderately low prediction accuracy._



# LOL

_A plausible but highly unlikely reason for this could be that the data could contain highly correlated attributes such as a, which may degrade the performance of supervised Naive Bayes classification, because the conditional independence assumption does not hold anymore._

_Even though the probability estimates of Naive Bayes are of low quality, its classification decisions are surprisingly good. NB classification usually has a much larger probability than the other classes and the estimates diverge very significantly from the true probabilities. Byt tge ckassufucatuin decusuib us vased ib wgucg class gets the highest score. It does not matter how accurate the estimates are. Correct estimation implies accurate prediction, but accurate prediction does not imply correct estimation._